# Florence-2 Large - Experimento com modelo de visão da Microsoft


---
---


Este notebook demonstra o uso do modelo **Florence-2 Large** da Microsoft para extração de texto de imagens através de OCR

## Sobre o experimento

- **Modelo**: microsoft/Florence-2-large
- **Objetivo**: Realizar OCR (Optical Character Recognition) em imagens e documentos
- **Ambiente**: Google Colab com GPU T4

## Configuração inicial

Primeiro, vamos instalar todas as dependências necessárias usando o `uv`


In [ ]:
# Install uv package manager
!pip install -q uv


### Instalação das dependências principais

Agora vamos instalar todas as bibliotecas necessárias:
- **PyTorch**: Framework de deep learning
- **Transformers**: Biblioteca da Hugging Face para modelos de linguagem
- **Pillow**: Para processamento de imagens


In [ ]:
# Install core dependencies with specific versions
!uv pip install transformers==4.49.0 pillow torch


---


## Importação das bibliotecas

Agora vamos importar todas as bibliotecas necessárias para o experimento


In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image

import torch
import time
import os


---


## Definição da função de inferência

Esta função processa a imagem e executa a tarefa de OCR usando o modelo Florence-2


In [ ]:
def run_example(processor, model, task_prompt, image_file):
    '''
    Executa a inferencia do modelo Florence-2 para uma determinada tarefa.
    
    Args:
        processor: Processador do modelo Florence-2
        model: Modelo Florence-2
        task_prompt: Prompt da tarefa (ex: '<OCR>')
        image_file: Objeto PIL Image
    
    Returns:
        Resultado parseado da inferencia
    '''

    inputs = processor(
        text = task_prompt,
        images = image_file,
        return_tensors = 'pt'
    ).to('cuda', torch.float16)

    generated_ids = model.generate(
        input_ids = inputs['input_ids'].cuda(),
        pixel_values = inputs['pixel_values'].cuda(),
        max_new_tokens = 1024,
        early_stopping = False,
        do_sample = False,
        num_beams = 3,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens = False)[0]

    parsed_answer = processor.post_process_generation(
        generated_text,
        task = task_prompt,
        image_size = (image_file.width, image_file.height)
    )

    return parsed_answer


---


## Carregando o modelo

Vamos carregar o modelo **Florence-2 Large** da Microsoft e o processador correspondente

In [ ]:
# Model ID: Florence-2 Large
model_id = 'microsoft/Florence-2-large'

# If you want to use the fine-tuned model, uncomment the following line
# model_id = 'microsoft/Florence-2-large-ft'

print('=' * 70)
print('CARREGANDO MODELO FLORENCE-2 LARGE')
print('=' * 70)
print(f'\nModelo: {model_id}')

print('\nCarregando modelo...')
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    torch_dtype = 'auto'
).eval().cuda()
print('Modelo carregado com sucesso!')

print('\nCarregando processador...')
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code = True
)
print('Processador carregado com sucesso!')

print('\n' + '=' * 70)
print(f'Dispositivo: {next(model.parameters()).device}')
print(f'Tipo de dados: {next(model.parameters()).dtype}')
print('=' * 70 + '\n')

---


## Configuração dos parâmetros

Vamos configurar os caminhos das imagens e os parâmetros para o processamento de OCR


In [ ]:
# Task prompt for OCR
task_prompt = '<OCR>'

# File paths
base_path = ''
image_file = os.path.join(base_path, 'data_experiments', 'case_01_drivers_license.jpeg')

print('CONFIGURAÇÃO DO EXPERIMENTO')
print('=' * 70)
print(f'Tarefa: {task_prompt}')
print(f'\nArquivo de entrada: {image_file}')
print('=' * 70 + '\n')


---


## Processamento de OCR

Agora vamos processar a imagem e extrair o texto usando o modelo Florence-2


In [ ]:
print('\n' + '=' * 70)
print('INICIANDO PROCESSAMENTO DE OCR')
print('=' * 70 + '\n')

# Process the image
if not os.path.exists(image_file):
    print(f'Erro: arquivo de imagem não encontrado: {image_file}')
else:
    print(f'Processando imagem: {os.path.basename(image_file)}')
    print('-' * 70)
    
    try:
        # Load the image
        image_file = Image.open(image_file).convert('RGB')

        print('\nExecutando inferência do modelo...')
        start_time = time.time()

        # Run OCR inference
        result = run_example(processor, model, task_prompt, image_file)

        end_time = time.time()
        inference_time = end_time - start_time

        print(f'Inferência concluída!')
        print(f'Tempo de processamento: {inference_time:.2f} segundos')

        print('\n' + '=' * 70)
        print('TEXTO EXTRAÍDO')
        print('=' * 70)
        print(result)
        print('=' * 70)

    except Exception as e:
        print(f'\nErro ao processar {os.path.basename(image_file)}: {str(e)}')
        import traceback
        traceback.print_exc()

print('\n' + '=' * 70)
print('PROCESSAMENTO CONCLUÍDO!')
print('=' * 70)


---


## Informações adicionais

### Sobre o Modelo Florence-2

O **Florence-2** é um modelo de visão e linguagem da Microsoft capaz de realizar múltiplas tarefas de visão computacional, incluindo:

- OCR (Optical Character Recognition)
- Detecção de objetos
- Segmentação
- Captioning de imagens
- Visual Question Answering

### Prompts Disponíveis

O modelo Florence-2 suporta diversos prompts de tarefa. Para OCR, o prompt deve ser exatamente:

```
<OCR>
```

Outros prompts disponiveis (para diferentes tarefas):
- `<CAPTION>`: gerar descrição da imagem
- `<DETAILED_CAPTION>`: gerar descrição detalhada
- `<MORE_DETAILED_CAPTION>`: gerar descrição muito detalhada
- `<CAPTION_TO_PHRASE_GROUNDING>`: captioning para ancoragem de frases (para esta tarefa, é necessário fornecer um contexto adicional para o modelo)
- `<OD>`: object detection
- `<DENSE_REGION_CAPTION>`: captioning denso de regiões
- `<REGION_PROPOSAL>`: propostas de regiões
- `<OCR_WITH_REGION>`: OCR com regiões específicas

### Configuração do modelo

- **max_new_tokens**: 1024 - número máximo de tokens gerados
- **num_beams**: 3 - Beam search para melhor qualidade
- **early_stopping**: False - Continua geração até o fim
- **do_sample**: False - Geração determinística

### Requisitos recomendados de Hardware

- GPU NVIDIA com CUDA
- Aproximadamente 8-12GB de VRAM para o modelo completo
